In [7]:
!pip install tk

In [39]:
!pip install qrcode[pil]

                                              0.0/45.7 kB ? eta -:--:--
     --------------------------               30.7/45.7 kB 1.4 MB/s eta 0:00:01
     -------------------------------------- 45.7/45.7 kB 572.1 kB/s eta 0:00:00


In [53]:
import sqlite3
import tkinter as tk
from tkinter import ttk, messagebox
from PIL import Image, ImageTk
import qrcode

# Connect to SQLite database
conn = sqlite3.connect('App_FoodOrderingfinal.db')
cursor = conn.cursor()

# Create main app window
root = tk.Tk()
root.title("Foodies Delight - Restaurant Management System")
root.geometry("900x750")
root.configure(bg="#f0f8ff")

# Function to switch between pages
def show_frame(frame):
    frame.tkraise()

# Create frames for pages
order_page = tk.Frame(root, bg="#f0f8ff")
payment_page = tk.Frame(root, bg="#f0f8ff")
for frame in (order_page, payment_page):
    frame.grid(row=0, column=0, sticky="nsew")

# --- ORDER PAGE ---
order_title = tk.Label(order_page, text="Foodies Delight - Order Page", font=("Helvetica", 24, "bold"), bg="#ffcccb", fg="#4b0082")
order_title.pack(pady=10)

restaurant_combo = ttk.Combobox(order_page, state="readonly")
restaurant_combo.set("Select a Restaurant")
restaurant_combo.pack(pady=10)
restaurant_combo.bind("<<ComboboxSelected>>", lambda e: load_menu(restaurant_combo.get()))

menu_tree = ttk.Treeview(order_page, columns=("Item ID", "Name", "Price"), show="headings")
menu_tree.heading("Item ID", text="Item ID")
menu_tree.heading("Name", text="Name")
menu_tree.heading("Price", text="Price")
menu_tree.pack(pady=10)

quantity_spinbox = tk.Spinbox(order_page, from_=1, to=20)
quantity_spinbox.pack(pady=5)

add_button = tk.Button(order_page, text="Add to Order", command=add_to_order, bg="#ff6347", fg="white")
add_button.pack(pady=5)

order_listbox = tk.Listbox(order_page, bg="#f5f5dc")
order_listbox.pack(pady=10)

total_label = tk.Label(order_page, text="Total Bill: $0.00", font=("Helvetica", 16), bg="#f0f8ff")
total_label.pack(pady=10)

customer_name_entry = tk.Entry(order_page)
customer_name_entry.insert(0, "Customer Name")
customer_name_entry.pack(pady=5)

table_no_entry = tk.Entry(order_page)
table_no_entry.insert(0, "Table Number")
table_no_entry.pack(pady=5)

next_button = tk.Button(order_page, text="Proceed to Payment", command=lambda: show_frame(payment_page), bg="#32cd32", fg="white")
next_button.pack(pady=10)

# --- PAYMENT PAGE ---
payment_title = tk.Label(payment_page, text="Foodies Delight - Payment & Feedback", font=("Helvetica", 24, "bold"), bg="#ffcccb", fg="#4b0082")
payment_title.pack(pady=10)

tk.Label(payment_page, text="Order Number:", bg="#f0f8ff").pack(pady=5)
order_number_label = tk.Label(payment_page, text="1", bg="#f0f8ff", font=("Helvetica", 16))
order_number_label.pack(pady=5)

payment_method_combo = ttk.Combobox(payment_page, state="readonly", values=["Credit Card", "Cash", "Digital Wallet"])
payment_method_combo.set("Select Payment Method")
payment_method_combo.pack(pady=5)

feedback_button = tk.Button(payment_page, text="Leave Feedback", command=leave_feedback, bg="#4682b4", fg="white")
feedback_button.pack(pady=5)

pay_button = tk.Button(payment_page, text="Pay & Generate QR Code", command=generate_qr_code, bg="#ff69b4", fg="white")
pay_button.pack(pady=10)

back_button = tk.Button(payment_page, text="Back to Order Page", command=lambda: show_frame(order_page), bg="#6a5acd", fg="white")
back_button.pack(pady=5)

# Sales Report Button
sales_report_button = tk.Button(payment_page, text="Generate Sales Report", command=generate_sales_report, bg="#ff8c00", fg="white")
sales_report_button.pack(pady=10)

# --- FUNCTIONS ---
def load_restaurants():
    cursor.execute("SELECT restaurant_id, name FROM Restaurants")
    restaurants = cursor.fetchall()
    restaurant_combo['values'] = [res[1] for res in restaurants]

def load_menu(restaurant_name):
    global selected_restaurant_id
    cursor.execute("SELECT restaurant_id FROM Restaurants WHERE name = ?", (restaurant_name,))
    selected_restaurant_id = cursor.fetchone()[0]
    cursor.execute("SELECT item_id, name, price FROM MenuItems WHERE restaurant_id = ?", (selected_restaurant_id,))
    items = cursor.fetchall()
    menu_tree.delete(*menu_tree.get_children())
    for item in items:
        menu_tree.insert('', 'end', values=(item[0], item[1], f"${item[2]:.2f}"))

def add_to_order():
    global total_bill
    selected_item = menu_tree.focus()
    if selected_item:
        item_details = menu_tree.item(selected_item, 'values')
        quantity = int(quantity_spinbox.get())
        price = float(item_details[2][1:]) * quantity
        total_bill += price
        selected_items.append((int(item_details[0]), item_details[1], float(item_details[2][1:]), quantity))
        order_listbox.insert(tk.END, f"{item_details[1]} - {quantity} pcs - ${price:.2f}")
        total_label.config(text=f"Total Bill: ${total_bill:.2f}")

def generate_qr_code():
    qr_data = f"Payment for Order #{order_number_label.cget('text')}: ${total_bill:.2f}"
    qr = qrcode.make(qr_data)
    qr.save("payment_qr.png")

    qr_window = tk.Toplevel(root)
    qr_window.title("Digital Wallet Payment QR Code")
    qr_window.geometry("300x300")

    qr_image = ImageTk.PhotoImage(Image.open("payment_qr.png"))
    qr_label = tk.Label(qr_window, image=qr_image)
    qr_label.image = qr_image
    qr_label.pack()

def generate_sales_report():
    cursor.execute("SELECT r.name, COUNT(o.order_id) AS total_orders, SUM(m.price * o.quantity) AS total_revenue FROM Orders o JOIN Restaurants r ON o.restaurant_id = r.restaurant_id JOIN MenuItems m ON o.item_id = m.item_id GROUP BY r.name")
    report_data = cursor.fetchall()

    report_window = tk.Toplevel(root)
    report_window.title("Sales Report")
    report_window.geometry("400x300")

    report_text = tk.Text(report_window, wrap=tk.WORD)
    report_text.pack(expand=True, fill=tk.BOTH)

    report_text.insert(tk.END, "Sales Report\n\n")
    for row in report_data:
        report_text.insert(tk.END, f"Restaurant: {row[0]}\n")
        report_text.insert(tk.END, f"Total Orders: {row[1]}\n")
        report_text.insert(tk.END, f"Total Revenue: ${row[2]:.2f}\n\n")

# Load initial data
load_restaurants()
show_frame(order_page)

# Run the app
root.mainloop()
